# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
from inauguralproject import householdClass

# Question 1

**Explanation of the code**

We start off by defining the different values for alpha and sigma in a list. Then we create empty lists, which are used to store the results when we solve the model, for these different values.

We construct a loop. This loop runs and solves the model, for all the different combinations of alpha and sigma. Then we calculate the ratio, HF/HM, and store these results in the list.

Finally, the results are plotted with matplotlib in a scatterplot.

In [ ]:
# Defining the different values of alpha and sigma
alpha_vals = [0.25, 0.5, 0.75]
sigma_vals = [0.5, 1.0, 1.5]

# Creating empty lists, to store results
ratios = []
alphas = []
sigmas = []

# Creating a loop for the different values of alpha and sigma
for alpha in alpha_vals:
    for sigma in sigma_vals:
        
        # Using the model to calculate optimal HF and HM
        household = householdClass()
        
        # Set the desired parameter values
        household.par.alpha = alpha
        household.par.sigma = sigma
        
        # Solving for the optimal values
        opt = household.solve_discrete()
        
        # Calculating the ratio, HF/HM
        ratio = opt.HF / opt.HM
        
        # Storing the results in the empty lists
        ratios.append(ratio)
        alphas.append(alpha)
        sigmas.append(sigma)
        
# Creating a scatterplot of the stored results in the list
plt.scatter(alphas, sigmas, c=ratios)
plt.colorbar(label='HF/HM ratio')
plt.xlabel('Alpha')
plt.ylabel('Sigma')
plt.show()


# Question 2

Explain your code and procedure

In [ ]:
#

# Question 3

Explain your code and procedure

In [ ]:
# code

ADD CONCISE CONLUSION.